In [ ]:
"""
Comprehensive tests for the RequestHandler class.
"""

import asyncio
import json
import pytest
import io
from unittest.mock import Mock, MagicMock, patch, AsyncMock, call, ANY

import aiohttp
import requests
from loguru import logger

from your_module.request_handler import RequestHandler


# ---------------------------------------------------------------------------
# Fixtures
# ---------------------------------------------------------------------------

@pytest.fixture
def request_handler():
    """Create a RequestHandler instance for testing."""
    return RequestHandler(
        base_url="https://api.example.com",
        default_timeout=10,
        long_timeout=60,
        max_retries=3,
        retry_delay=0.1,  # Short delay for tests
        semaphore_limit=5,
        verify_ssl=False,
    )


@pytest.fixture
def mock_response():
    """Create a mock for requests.Response."""
    response = Mock(spec=requests.Response)
    response.status_code = 200
    response.text = "Response text"
    response.json.return_value = {"key": "value"}
    return response


@pytest.fixture
def mock_file():
    """Create a mock file-like object."""
    return io.BytesIO(b"This is test file content")


@pytest.fixture
def mock_aiohttp_response():
    """Create a mock for aiohttp.ClientResponse."""
    response = AsyncMock(spec=aiohttp.ClientResponse)
    response.status = 200
    response.text = AsyncMock(return_value="Response text")
    response.json = AsyncMock(return_value={"key": "value"})
    response.read = AsyncMock(return_value=b'{"key": "value"}')
    response.content = AsyncMock()
    response.close = AsyncMock()
    
    # Setup streaming content
    async def mock_iter_chunked(chunk_size):
        yield b'{"streaming": true, "stream_data": {"text": "Streaming text"}}'
        yield b'{"streaming": true, "stream_data": {"text": "More text"}}'
        yield b'{"final_response": {"answer": "Final answer"}}'
    
    response.content.iter_chunked = mock_iter_chunked
    return response


@pytest.fixture
def mock_stream_response():
    """Create a mock for a streaming requests.Response."""
    response = Mock(spec=requests.Response)
    response.status_code = 200
    response.close = Mock()
    
    # Setup streaming content
    def mock_iter_content(chunk_size=None):
        yield b'{"streaming": true, "stream_data": {"text": "Streaming text"}}'
        yield b'{"streaming": true, "stream_data": {"text": "More text"}}'
        yield b'{"final_response": {"answer": "Final answer"}}'
    
    response.iter_content = mock_iter_content
    return response


@pytest.fixture
def mock_empty_stream_response():
    """Create a mock for an empty streaming response."""
    response = Mock(spec=requests.Response)
    response.status_code = 200
    response.close = Mock()
    
    # Empty stream
    def mock_iter_content(chunk_size=None):
        yield b''
    
    response.iter_content = mock_iter_content
    return response


@pytest.fixture
def mock_invalid_json_stream_response():
    """Create a mock for a streaming response with invalid JSON."""
    response = Mock(spec=requests.Response)
    response.status_code = 200
    response.close = Mock()
    
    # Invalid JSON in stream
    def mock_iter_content(chunk_size=None):
        yield b'{"streaming": true, "stream_data": {"text": "Text"}'  # Missing closing brace
        yield b', "more": "data"}'  # Continuation that makes it valid when combined
    
    response.iter_content = mock_iter_content
    return response


@pytest.fixture
def mock_session():
    """Create a mock aiohttp.ClientSession."""
    session = AsyncMock(spec=aiohttp.ClientSession)
    session.__aenter__.return_value = session
    session.__aexit__.return_value = None
    return session


# ---------------------------------------------------------------------------
# Initialization Tests
# ---------------------------------------------------------------------------

def test_init_defaults():
    """Test initialization with default parameters."""
    handler = RequestHandler(base_url="https://api.example.com")
    
    assert handler.base_url == "https://api.example.com"
    assert handler.default_timeout == 30  # Default value
    assert handler.long_timeout == 300  # Default value
    assert handler.max_retries == 5  # Default value
    assert handler.retry_delay == 5  # Default value
    assert handler.semaphore_limit == 50  # Default value
    assert handler.verify_ssl is False  # Default value
    assert isinstance(handler._semaphore, asyncio.Semaphore)
    assert handler._semaphore._value == 50


def test_init_custom_values():
    """Test initialization with custom parameters."""
    handler = RequestHandler(
        base_url="https://custom.api.com",
        default_timeout=15,
        long_timeout=120,
        max_retries=2,
        retry_delay=3,
        semaphore_limit=10,
        verify_ssl=True,
    )
    
    assert handler.base_url == "https://custom.api.com"
    assert handler.default_timeout == 15
    assert handler.long_timeout == 120
    assert handler.max_retries == 2
    assert handler.retry_delay == 3
    assert handler.semaphore_limit == 10
    assert handler.verify_ssl is True
    assert isinstance(handler._semaphore, asyncio.Semaphore)
    assert handler._semaphore._value == 10


# ---------------------------------------------------------------------------
# Synchronous HTTP Method Tests
# ---------------------------------------------------------------------------

@patch('requests.post')
def test_post(mock_post, request_handler, mock_response):
    """Test POST method returns response on success."""
    mock_post.return_value = mock_response
    
    response = request_handler.post('/endpoint', {"Content-Type": "application/json"}, {"data": "value"})
    
    assert response == mock_response
    mock_post.assert_called_once_with(
        'https://api.example.com/endpoint',
        headers={"Content-Type": "application/json"},
        json={"data": "value"},
        verify=False,
        timeout=60
    )


@patch('requests.post')
def test_post_with_files(mock_post, request_handler, mock_response, mock_file):
    """Test POST method with file uploads."""
    mock_post.return_value = mock_response
    files = {"file": ("test.txt", mock_file, "text/plain")}
    
    response = request_handler.post(
        '/upload', 
        {"Authorization": "Bearer token"}, 
        {"description": "Test file"}, 
        files
    )
    
    assert response == mock_response
    mock_post.assert_called_once_with(
        'https://api.example.com/upload',
        headers={"Authorization": "Bearer token"},
        data={"description": "Test file"},
        files=files,
        verify=False,
        timeout=60
    )


@patch('requests.post')
def test_post_with_custom_timeout(mock_post, request_handler, mock_response):
    """Test POST method with custom timeout."""
    mock_post.return_value = mock_response
    
    response = request_handler.post(
        '/endpoint', 
        {"Content-Type": "application/json"}, 
        {"data": "value"}, 
        timeout=30
    )
    
    assert response == mock_response
    mock_post.assert_called_once_with(
        'https://api.example.com/endpoint',
        headers={"Content-Type": "application/json"},
        json={"data": "value"},
        verify=False,
        timeout=30  # Custom timeout
    )


@patch('requests.get')
def test_get(mock_get, request_handler, mock_response):
    """Test GET method returns response on success."""
    mock_get.return_value = mock_response
    
    response = request_handler.get('/endpoint', {"Content-Type": "application/json"})
    
    assert response == mock_response
    mock_get.assert_called_once_with(
        'https://api.example.com/endpoint',
        headers={"Content-Type": "application/json"},
        verify=False,
        timeout=10  # Default timeout
    )


@patch('requests.get')
def test_get_with_custom_timeout(mock_get, request_handler, mock_response):
    """Test GET method with custom timeout."""
    mock_get.return_value = mock_response
    
    response = request_handler.get('/endpoint', {"Content-Type": "application/json"}, timeout=15)
    
    assert response == mock_response
    mock_get.assert_called_once_with(
        'https://api.example.com/endpoint',
        headers={"Content-Type": "application/json"},
        verify=False,
        timeout=15  # Custom timeout
    )


@patch('requests.put')
def test_put(mock_put, request_handler, mock_response):
    """Test PUT method returns response on success."""
    mock_put.return_value = mock_response
    
    response = request_handler.put('/endpoint', {"Content-Type": "application/json"}, {"data": "value"})
    
    assert response == mock_response
    mock_put.assert_called_once_with(
        'https://api.example.com/endpoint',
        headers={"Content-Type": "application/json"},
        json={"data": "value"},
        verify=False,
        timeout=10  # Default timeout
    )


@patch('requests.delete')
def test_delete(mock_delete, request_handler, mock_response):
    """Test DELETE method returns response on success."""
    mock_delete.return_value = mock_response
    
    response = request_handler.delete('/endpoint', {"Content-Type": "application/json"})
    
    assert response == mock_response
    mock_delete.assert_called_once_with(
        'https://api.example.com/endpoint',
        headers={"Content-Type": "application/json"},
        verify=False,
        timeout=10  # Default timeout
    )


@patch('requests.post')
def test_post_stream(mock_post, request_handler, mock_stream_response):
    """Test post_stream method returns response on success."""
    mock_post.return_value = mock_stream_response
    
    response = request_handler.post_stream('/endpoint', {"Content-Type": "application/json"}, {"data": "value"})
    
    assert response == mock_stream_response
    mock_post.assert_called_once_with(
        'https://api.example.com/endpoint',
        headers={"Content-Type": "application/json"},
        json={"data": "value"},
        stream=True,
        verify=False,
        timeout=60
    )


@patch('requests.post')
def test_post_stream_error_response(mock_post, request_handler):
    """Test post_stream method handles error responses."""
    error_response = Mock(spec=requests.Response)
    error_response.status_code = 404
    error_response.text = "Not found"
    error_response.close = Mock()
    mock_post.return_value = error_response
    
    response = request_handler.post_stream('/endpoint', {}, {})
    
    assert response is None
    error_response.close.assert_called_once()


# ---------------------------------------------------------------------------
# Asynchronous HTTP Method Tests
# ---------------------------------------------------------------------------

@pytest.mark.asyncio
async def test_async_post(request_handler, mock_aiohttp_response):
    """Test async_post method returns response on success."""
    with patch('aiohttp.ClientSession.post', new_callable=AsyncMock) as mock_post:
        mock_post.return_value = mock_aiohttp_response
        
        response = await request_handler.async_post('/endpoint', {"Content-Type": "application/json"}, {"data": "value"})
        
        assert response == mock_aiohttp_response
        mock_post.assert_called_once_with(
            'https://api.example.com/endpoint',
            headers={"Content-Type": "application/json"},
            json={"data": "value"},
            timeout=60,
            ssl=None
        )


@pytest.mark.asyncio
async def test_async_post_with_files(request_handler, mock_aiohttp_response, mock_file):
    """Test async_post method with file uploads."""
    with patch('aiohttp.ClientSession.post', new_callable=AsyncMock) as mock_post:
        mock_post.return_value = mock_aiohttp_response
        files = {"file": ("test.txt", mock_file, "text/plain")}
        
        response = await request_handler.async_post(
            '/upload', 
            {"Authorization": "Bearer token"}, 
            {"description": "Test file"}, 
            files
        )
        
        assert response == mock_aiohttp_response
        mock_post.assert_called_once()
        # Check that FormData was passed (can't easily check exact contents)
        args, kwargs = mock_post.call_args
        assert 'data' in kwargs
        assert isinstance(kwargs['data'], aiohttp.FormData)


@pytest.mark.asyncio
async def test_async_post_with_custom_timeout(request_handler, mock_aiohttp_response):
    """Test async_post method with custom timeout."""
    with patch('aiohttp.ClientSession.post', new_callable=AsyncMock) as mock_post:
        mock_post.return_value = mock_aiohttp_response
        
        response = await request_handler.async_post(
            '/endpoint', 
            {"Content-Type": "application/json"}, 
            {"data": "value"}, 
            timeout=30
        )
        
        assert response == mock_aiohttp_response
        mock_post.assert_called_once_with(
            'https://api.example.com/endpoint',
            headers={"Content-Type": "application/json"},
            json={"data": "value"},
            timeout=30,  # Custom timeout
            ssl=None
        )


@pytest.mark.asyncio
async def test_async_get(request_handler, mock_aiohttp_response):
    """Test async_get method returns response on success."""
    with patch('aiohttp.ClientSession.get', new_callable=AsyncMock) as mock_get:
        mock_get.return_value = mock_aiohttp_response
        
        response = await request_handler.async_get('/endpoint', {"Content-Type": "application/json"})
        
        assert response == mock_aiohttp_response
        mock_get.assert_called_once_with(
            'https://api.example.com/endpoint',
            headers={"Content-Type": "application/json"},
            timeout=10,
            ssl=None
        )


@pytest.mark.asyncio
async def test_async_put(request_handler, mock_aiohttp_response):
    """Test async_put method returns response on success."""
    with patch('aiohttp.ClientSession.put', new_callable=AsyncMock) as mock_put:
        mock_put.return_value = mock_aiohttp_response
        
        response = await request_handler.async_put('/endpoint', {"Content-Type": "application/json"}, {"data": "value"})
        
        assert response == mock_aiohttp_response
        mock_put.assert_called_once_with(
            'https://api.example.com/endpoint',
            headers={"Content-Type": "application/json"},
            json={"data": "value"},
            timeout=10,
            ssl=None
        )


@pytest.mark.asyncio
async def test_async_delete(request_handler, mock_aiohttp_response):
    """Test async_delete method returns response on success."""
    with patch('aiohttp.ClientSession.delete', new_callable=AsyncMock) as mock_delete:
        mock_delete.return_value = mock_aiohttp_response
        
        response = await request_handler.async_delete('/endpoint', {"Content-Type": "application/json"})
        
        assert response == mock_aiohttp_response
        mock_delete.assert_called_once_with(
            'https://api.example.com/endpoint',
            headers={"Content-Type": "application/json"},
            timeout=10,
            ssl=None
        )


@pytest.mark.asyncio
async def test_async_post_stream(request_handler, mock_aiohttp_response):
    """Test async_post_stream method returns response on success."""
    with patch('aiohttp.ClientSession', new_callable=AsyncMock) as mock_session_class:
        session_instance = AsyncMock()
        session_instance.post.return_value = mock_aiohttp_response
        mock_session_class.return_value = session_instance
        mock_session_class.return_value.__aenter__.return_value = session_instance
        
        response = await request_handler.async_post_stream(
            '/endpoint', 
            {"Content-Type": "application/json"}, 
            {"data": "value"},
            chunk_size=8192
        )
        
        assert response == mock_aiohttp_response
        assert hasattr(response, "_chunk_size")
        assert response._chunk_size == 8192
        session_instance.post.assert_called_once_with(
            'https://api.example.com/endpoint',
            headers={"Content-Type": "application/json"},
            json={"data": "value"},
            ssl=None,
            raise_for_status=False
        )


@pytest.mark.asyncio
async def test_async_post_stream_error_response(request_handler):
    """Test async_post_stream method handles error responses."""
    with patch('aiohttp.ClientSession', new_callable=AsyncMock) as mock_session_class:
        session_instance = AsyncMock()
        error_response = AsyncMock()
        error_response.status = 404
        error_response.text = AsyncMock(return_value="Not found")
        
        session_instance.post.return_value = error_response
        mock_session_class.return_value = session_instance
        mock_session_class.return_value.__aenter__.return_value = session_instance
        
        response = await request_handler.async_post_stream('/endpoint', {}, {})
        
        assert response is None
        session_instance.close.assert_called_once()


@pytest.mark.asyncio
async def test_async_post_stream_exception(request_handler):
    """Test async_post_stream method handles exceptions."""
    with patch('aiohttp.ClientSession', new_callable=AsyncMock) as mock_session_class:
        session_instance = AsyncMock()
        session_instance.post.side_effect = Exception("Connection error")
        mock_session_class.return_value = session_instance
        mock_session_class.return_value.__aenter__.return_value = session_instance
        
        response = await request_handler.async_post_stream('/endpoint', {}, {})
        
        assert response is None
        session_instance.close.assert_called_once()


# ---------------------------------------------------------------------------
# Semaphore Tests
# ---------------------------------------------------------------------------

@pytest.mark.asyncio
async def test_semaphore_limit_respected(request_handler, mock_aiohttp_response):
    """Test that semaphore limit is respected for async requests."""
    # Override semaphore for testing
    request_handler._semaphore = AsyncMock(spec=asyncio.Semaphore)
    request_handler._semaphore.__aenter__ = AsyncMock()
    request_handler._semaphore.__aexit__ = AsyncMock()
    
    with patch('aiohttp.ClientSession.get', new_callable=AsyncMock) as mock_get:
        mock_get.return_value = mock_aiohttp_response
        
        await request_handler.async_get('/endpoint', {})
        
        # Verify semaphore was acquired and released
        request_handler._semaphore.__aenter__.assert_called_once()
        request_handler._semaphore.__aexit__.assert_called_once()


# ---------------------------------------------------------------------------
# Response Processing Tests
# ---------------------------------------------------------------------------

def test_process_response_json(request_handler, mock_response):
    """Test process_response handles JSON responses."""
    result = request_handler.process_response(mock_response)
    
    assert result == {"key": "value"}
    mock_response.json.assert_called_once()


def test_process_response_text(request_handler, mock_response):
    """Test process_response falls back to text when JSON fails."""
    mock_response.json.side_effect = ValueError("Invalid JSON")
    
    result = request_handler.process_response(mock_response)
    
    assert result == "Response text"


def test_process_response_none(request_handler):
    """Test process_response handles None input."""
    result = request_handler.process_response(None)
    
    assert result is None


def test_process_response_no_json_or_text(request_handler):
    """Test process_response when response has neither json nor text."""
    response = Mock()
    response.json.side_effect = AttributeError("No json method")
    
    # No text attribute
    result = request_handler.process_response(response)
    
    assert result == response  # Returns response as is


@pytest.mark.asyncio
async def test_process_async_response_json(request_handler, mock_aiohttp_response):
    """Test process_async_response handles JSON responses."""
    result = await request_handler.process_async_response(mock_aiohttp_response)
    
    assert result == {"key": "value"}
    mock_aiohttp_response.json.assert_called_once()


@pytest.mark.asyncio
async def test_process_async_response_text(request_handler, mock_aiohttp_response):
    """Test process_async_response falls back to text when JSON fails."""
    mock_aiohttp_response.json.side_effect = ValueError("Invalid JSON")
    
    result = await request_handler.process_async_response(mock_aiohttp_response)
    
    assert result == "Response text"
    mock_aiohttp_response.text.assert_called_once()


@pytest.mark.asyncio
async def test_process_async_response_all_fails(request_handler, mock_aiohttp_response):
    """Test process_async_response when both json and text fail."""
    mock_aiohttp_response.json.side_effect = ValueError("Invalid JSON")
    mock_aiohttp_response.text.side_effect = Exception("Text error")
    
    result = await request_handler.process_async_response(mock_aiohttp_response)
    
    assert result is None


# ---------------------------------------------------------------------------
# Streaming Tests
# ---------------------------------------------------------------------------

def test_process_stream_response(request_handler, mock_stream_response):
    """Test process_stream_response extracts final answer."""
    with patch('builtins.print') as mock_print:
        result = request_handler.process_stream_response(mock_stream_response)
        
        assert result == "Final answer"
        assert mock_print.call_count == 2  # Two streaming messages
        mock_print.assert_has_calls([
            call("Streaming text", end=""),
            call("More text", end="")
        ])
        mock_stream_response.close.assert_called_once()


def test_process_stream_response_empty(request_handler, mock_empty_stream_response):
    """Test process_stream_response with empty stream."""
    result = request_handler.process_stream_response(mock_empty_stream_response)
    
    assert result is None
    mock_empty_stream_response.close.assert_called_once()


def test_process_stream_response_invalid_json(request_handler, mock_invalid_json_stream_response):
    """Test process_stream_response with invalid JSON that becomes valid."""
    result = request_handler.process_stream_response(mock_invalid_json_stream_response)
    
    # Should still parse successfully when chunks are combined
    assert result is None  # No final_response in this mock
    mock_invalid_json_stream_response.close.assert_called_once()


def test_process_stream_response_exception(request_handler, mock_stream_response):
    """Test process_stream_response handles exceptions."""
    mock_stream_response.iter_content.side_effect = Exception("Connection error")
    
    result = request_handler.process_stream_response(mock_stream_response)
    
    assert result is None
    mock_stream_response.close.assert_called_once()


@pytest.mark.asyncio
async def test_process_async_stream_response(request_handler, mock_aiohttp_response):
    """Test process_async_stream_response extracts final answer."""
    with patch('builtins.print') as mock_print:
        result = await request_handler.process_async_stream_response(mock_aiohttp_response)
        
        assert result == "Final answer"
        assert mock_print.call_count == 2  # Two streaming messages
        mock_print.assert_has_calls([
            call("Streaming text", end=""),
            call("More text", end="")
        ])


@pytest.mark.asyncio
async def test_process_async_stream_response_session_cleanup(request_handler, mock_aiohttp_response):
    """Test that the session is closed after processing."""
    session = AsyncMock()
    mock_aiohttp_response._session = session
    
    await request_handler.process_async_stream_response(mock_aiohttp_response)
    
    session.close.assert_called_once()


@pytest.mark.asyncio
async def test_process_async_stream_response_custom_chunk_size(request_handler, mock_aiohttp_response):
    """Test that custom chunk size is used when provided."""
    mock_aiohttp_response._chunk_size = 16384
    
    with patch.object(mock_aiohttp_response.content, 'iter_chunked') as mock_iter_chunked:
        mock_iter_chunked.return_value = aiter([
            b'{"final_response": {"answer": "Final answer"}}'
        ])
        
        await request_handler.process_async_stream_response(mock_aiohttp_response)
        
        mock_iter_chunked.assert_called_once_with(16384)


@pytest.mark.asyncio
async def test_process_async_stream_response_no_chunk_size(request_handler, mock_aiohttp_response):
    """Test that default chunk size is used when not provided."""
    # Ensure no _chunk_size attribute
    if hasattr(mock_aiohttp_response, '_chunk_size'):
        delattr(mock_aiohttp_response, '_chunk_size')
    
    with patch.object(mock_aiohttp_response.content, 'iter_chunked') as mock_iter_chunked:
        mock_iter_chunked.return_value = aiter([
            b'{"final_response": {"answer": "Final answer"}}'
        ])
        
        await request_handler.process_async_stream_response(mock_aiohttp_response)
        
        mock_iter_chunked.assert_called_once_with(4096)  # Default value


@pytest.mark.asyncio
async def test_simple_async_stream(request_handler, mock_aiohttp_response):
    """Test simple_async_stream combines post and process."""
    with patch.object(request_handler, 'async_post_stream', new_callable=AsyncMock) as mock_post_stream:
        with patch.object(request_handler, 'process_async_stream_response', new_callable=AsyncMock) as mock_process:
            mock_post_stream.return_value = mock_aiohttp_response
            mock_process.return_value = "Final answer"
            
            result = await request_handler.simple_async_stream('/endpoint', {}, {}, timeout=45, chunk_size=8192)
            
            assert result == "Final answer"
            mock_post_stream.assert_called_once_with(
                path='/endpoint', 
                headers={}, 
                payload={}, 
                timeout=45,
                chunk_size=8192
            )
            mock_process.assert_called_with(mock_aiohttp_response)


@pytest.mark.asyncio
async def test_simple_async_stream_failure(request_handler):
    """Test simple_async_stream when async_post_stream fails."""
    with patch.object(request_handler, 'async_post_stream', new_callable=AsyncMock) as mock_post_stream:
        with patch.object(request_handler, 'process_async_stream_response', new_callable=AsyncMock) as mock_process:
            mock_post_stream.return_value = None
            
            result = await request_handler.simple_async_stream('/endpoint', {}, {})
            
            assert result is None
            mock_post_stream.assert_called_once()
            mock_process.assert_not_called()


@pytest.mark.asyncio
async def test_stream_chunks(request_handler, mock_aiohttp_response):
    """Test stream_chunks yields JSON objects."""
    callback_mock = Mock()
    
    results = []
    async for json_data in request_handler.stream_chunks(mock_aiohttp_response, callback_mock):
        results.append(json_data)
    
    assert len(results) == 3
    assert results[0] == {"streaming": True, "stream_data": {"text": "Streaming text"}}
    assert results[1] == {"streaming": True, "stream_data": {"text": "More text"}}
    assert results[2] == {"final_response": {"answer": "Final answer"}}
    
    callback_mock.assert_has_calls([
        call("Streaming text"),
        call("More text")
    ])


@pytest.mark.asyncio
async def test_stream_chunks_none_response(request_handler):
    """Test stream_chunks handles None response."""
    results = []
    async for json_data in request_handler.stream_chunks(None):
        results.append(json_data)
    
    assert len(results) == 0


@pytest.mark.asyncio
async def test_stream_chunks_session_cleanup(request_handler, mock_aiohttp_response):
    """Test that the session is closed after streaming."""
    session = AsyncMock()
    mock_aiohttp_response._session = session
    
    results = []
    async for _ in request_handler.stream_chunks(mock_aiohttp_response):
        pass
    
    session.close.assert_called_once()


@pytest.mark.asyncio
async def test_stream_chunks_exception_during_iteration(request_handler, mock_aiohttp_response):
    """Test stream_chunks handles exceptions during iteration."""
    mock_aiohttp_response.content.iter_chunked.side_effect = Exception("Connection error")
    session = AsyncMock()
    mock_aiohttp_response._session = session
    
    results = []
    async for json_data in request_handler.stream_chunks(mock_aiohttp_response):
        results.append(json_data)
    
    assert len(results) == 0
    session.close.assert_called_once()


# ---------------------------------------------------------------------------
# Retry Logic Tests
# ---------------------------------------------------------------------------

@patch('requests.post')
def test_post_with_retry_success(mock_post, request_handler, mock_response):
    """Test _post_with_retry succeeds on first try."""
    mock_post.return_value = mock_response
    
    response = request_handler._post_with_retry('/endpoint', {}, {})
    
    assert response == mock_response
    assert mock_post.call_count == 1


@patch('requests.post')
def test_post_with_retry_fails_then_succeeds(mock_post, request_handler, mock_response):
    """Test _post_with_retry retries after failure."""
    # First call raises exception, second call succeeds
    mock_post.side_effect = [Exception("Connection error"), mock_response]
    
    response = request_handler._post_with_retry('/endpoint', {}, {})
    
    assert response == mock_response
    assert mock_post.call_count == 2


@patch('requests.post')
def test_post_with_retry_all_fails(mock_post, request_handler):
    """Test _post_with_retry returns None after all retries fail."""
    # All calls raise exception
    mock_post.side_effect = Exception("Connection error")
    
    response = request_handler._post_with_retry('/endpoint', {}, {}, retries=3)
    
    assert response is None
    assert mock_post.call_count == 3


@patch('requests.post')
def test_post_with_retry_4xx_no_retry(mock_post, request_handler, mock_response):
    """Test _post_with_retry doesn't retry on 4xx errors."""
    mock_response.status_code = 404
    mock_post.return_value = mock_response
    
    response = request_handler._post_with_retry('/endpoint', {}, {})
    
    assert response == mock_response
    assert mock_post.call_count == 1


@patch('requests.post')
def test_post_with_retry_5xx_retry(mock_post, request_handler, mock_response):
    """Test _post_with_retry retries on 5xx errors."""
    error_response = Mock(spec=requests.Response)
    error_response.status_code = 500
    error_response.text = "Server error"
    
    # First call returns 5xx, second call succeeds
    mock_post.side_effect = [error_response, mock_response]
    
    response = request_handler._post_with_retry('/endpoint', {}, {})
    
    assert response == mock_response
    assert mock_post.call_count == 2


@pytest.mark.asyncio
async def test_async_post_with_retry_success(request_handler, mock_aiohttp_response):
    """Test _async_post_with_retry succeeds on first try."""
    with patch('aiohttp.ClientSession.post', new_callable=AsyncMock) as mock_post:
        mock_post.return_value = mock_aiohttp_response
        
        response = await request_handler._async_post_with_retry('/endpoint', {}, {})
        
        assert response == mock_aiohttp_response
        assert mock_post.call_count == 1


@pytest.mark.asyncio
async def test_async_post_with_retry_fails_then_succeeds(request_handler, mock_aiohttp_response):
    """Test _async_post_with_retry retries after failure."""
    with patch('aiohttp.ClientSession.post', new_callable=AsyncMock) as mock_post:
        # First call raises exception, second call succeeds
        mock_post.side_effect = [Exception("Connection error"), mock_aiohttp_response]
        
        response = await request_handler._async_post_with_retry('/endpoint', {}, {})
        
        assert response == mock_aiohttp_response
        assert mock_post.call_count == 2


@pytest.mark.asyncio
async def test_async_post_with_retry_5xx_retry(request_handler, mock_aiohttp_response):
    """Test _async_post_with_retry retries on 5xx errors."""
    with patch('aiohttp.ClientSession.post', new_callable=AsyncMock) as mock_post:
        error_response = AsyncMock()
        error_response.status = 500
        error_response.text = AsyncMock(return_value="Server error")
        
        # First call returns 5xx, second call succeeds
        mock_post.side_effect = [error_response, mock_aiohttp_response]
        
        response = await request_handler._async_post_with_retry('/endpoint', {}, {})
        
        assert response == mock_aiohttp_response
        assert mock_post.call_count == 2


@pytest.mark.asyncio
async def test_async_post_with_retry_with_files(request_handler, mock_aiohttp_response, mock_file):
    """Test _async_post_with_retry with file uploads."""
    with patch('aiohttp.ClientSession.post', new_callable=AsyncMock) as mock_post:
        mock_post.return_value = mock_aiohttp_response
        files = {"file": ("test.txt", mock_file, "text/plain")}
        
        response = await request_handler._async_post_with_retry('/upload', {}, {}, files)
        
        assert response == mock_aiohttp_response
        mock_post.assert_called_once()
        # Check that FormData was passed (can't easily check exact contents)
        args, kwargs = mock_post.call_args
        assert 'data' in kwargs
        assert isinstance(kwargs['data'], aiohttp.FormData)


# ---------------------------------------------------------------------------
# Special Cases
# ---------------------------------------------------------------------------

@patch('requests.post')
def test_verify_ssl_true(mock_post, mock_response):
    """Test verify_ssl=True is passed to requests."""
    handler = RequestHandler(base_url="https://api.example.com", verify_ssl=True)
    mock_post.return_value = mock_response
    
    handler.post('/endpoint', {}, {})
    
    args, kwargs = mock_post.call_args
    assert kwargs['verify'] is True


@pytest.mark.asyncio
async def test_async_verify_ssl_true(mock_aiohttp_response):
    """Test verify_ssl=True is passed to aiohttp."""
    handler = RequestHandler(base_url="https://api.example.com", verify_ssl=True)
    
    with patch('aiohttp.ClientSession.get', new_callable=AsyncMock) as mock_get:
        mock_get.return_value = mock_aiohttp_response
        
        await handler.async_get('/endpoint', {})
        
        args, kwargs = mock_get.call_args
        assert kwargs['ssl'] is True


if __name__ == "__main__":
    pytest.main()